In [1]:
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns #Control figure 
import numpy as np
import os
matplotlib.style.use('ggplot')
%matplotlib inline

C:\Users\Elizabeth\Anaconda3\lib\site-packages\statsmodels\tools\_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [2]:
from sodapy import Socrata

In [3]:
#MyAppToken = ''

In [4]:
#client = Socrata("data.cityofnewyork.us", MyAppToken)

In [5]:
#results = client.get("dsg6-ifza", limit=2000)

In [6]:
#df = pd.DataFrame.from_records(results)

In [7]:
cwd = os.getcwd() #to get current working directory
#print(cwd)

In [8]:
df = pd.read_csv('DOHMH_Childcare_Center_Inspections.csv', encoding = "L1")

In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 52024 entries, 0 to 52023
Data columns (total 34 columns):
 #   Column                                       Non-Null Count  Dtype  
---  ------                                       --------------  -----  
 0   Center Name                                  52024 non-null  object 
 1   Legal Name                                   52024 non-null  object 
 2   Building                                     52013 non-null  object 
 3   Street                                       52024 non-null  object 
 4   Borough                                      52024 non-null  object 
 5   ZipCode                                      51942 non-null  float64
 6   Phone                                        51977 non-null  object 
 7   Permit Number                                46227 non-null  float64
 8   Permit Expiration                            52024 non-null  object 
 9   Status                                       52024 non-null  object 
 10

In [10]:
def clean_string(astr):
    return astr.lower().replace('.', '') \
        .replace(',', '') \
        .replace(';', '') \
        .replace(':', '') \
        .replace('á', 'a') \
        .replace('é', 'e') \
        .replace('í', 'i') \
        .replace('ó', 'o') \
        .replace('ú', 'u') \
        .replace(' ', '_') \
        .replace('ñ', 'ni')

In [11]:
def clean_columns(df):
    for series in df:
        df.rename(columns={series:clean_string(series)}, inplace=True)

In [12]:
def execute(raw_dataset_path, clean_dataset_path):
    print("\t-> Leyendo datos crudos a un DataFrame")
    df = pd.read_csv('DOHMH_Childcare_Center_Inspections.csv')

In [13]:
print("\t-> Limpando columnas")

	-> Limpando columnas


In [14]:
clean_columns(df)

In [15]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 52024 entries, 0 to 52023
Data columns (total 34 columns):
 #   Column                                       Non-Null Count  Dtype  
---  ------                                       --------------  -----  
 0   center_name                                  52024 non-null  object 
 1   legal_name                                   52024 non-null  object 
 2   building                                     52013 non-null  object 
 3   street                                       52024 non-null  object 
 4   borough                                      52024 non-null  object 
 5   zipcode                                      51942 non-null  float64
 6   phone                                        51977 non-null  object 
 7   permit_number                                46227 non-null  float64
 8   permit_expiration                            52024 non-null  object 
 9   status                                       52024 non-null  object 
 10

In [16]:
print("\t-> Reemplazando espacios en blanco")
for col in df.select_dtypes('object'):
    df[col] = df[col].replace('\s+', ' ', regex=True)

	-> Reemplazando espacios en blanco


In [17]:
print("\t-> Limpiando valores")
for col in df.select_dtypes('object'):
        df[col] = df[col].str.strip()
        df[col] = df[col].str.lower()
        df[col] = df[col].str.replace('á', 'a')
        df[col] = df[col].str.replace('é', 'e')
        df[col] = df[col].str.replace('í', 'i')
        df[col] = df[col].str.replace('ó', 'o')
        df[col] = df[col].str.replace('ú', 'u')
        df[col] = df[col].str.replace(' ', '_')

	-> Limpiando valores


In [18]:
print("\t-> Cambiando NA por np.nan")
for col in df.select_dtypes('object'):
    df.loc[df[col] == 'na', col] = np.nan

	-> Cambiando NA por np.nan


In [19]:
df.to_csv('df.csv', index=False)

In [20]:
print("\t-> Cuántos valores NaN tiene la base")

	-> Cuántos valores NaN tiene la base


In [21]:
df.isnull().sum()

center_name                                        0
legal_name                                         0
building                                          11
street                                             0
borough                                            0
zipcode                                           82
phone                                             47
permit_number                                   5797
permit_expiration                                  0
status                                             0
age_range                                       2316
maximum_capacity                                   0
day_care_id                                        0
program_type                                       0
facility_type                                      0
child_care_type                                    0
building_identification_number                     0
url                                            26395
date_permitted                                

In [22]:
print("\t-> Eliminar duplicados")

	-> Eliminar duplicados


In [23]:
df.duplicated().sum()

12653

In [24]:
df = df.drop_duplicates()
df.shape

(39371, 34)

### TABLA 3

In [25]:
dummies = ["borough", "status", "program_type", "facility_type"]
df_1 = df[[i for i in df.keys() if i not in dummies]]
df_2 = pd.get_dummies(df[dummies])

In [26]:
df = pd.concat([df_1.reset_index(drop=True), df_2], axis=1)
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 48920 entries, 0 to 52023
Data columns (total 46 columns):
 #   Column                                       Non-Null Count  Dtype  
---  ------                                       --------------  -----  
 0   center_name                                  39371 non-null  object 
 1   legal_name                                   39371 non-null  object 
 2   building                                     39360 non-null  object 
 3   street                                       39371 non-null  object 
 4   zipcode                                      39321 non-null  float64
 5   phone                                        39325 non-null  object 
 6   permit_number                                35549 non-null  float64
 7   permit_expiration                            39371 non-null  object 
 8   age_range                                    37584 non-null  object 
 9   maximum_capacity                             39371 non-null  float64
 10

### TABLA 4

In [27]:
print("\t-> Reagrupar en tres variables Inspection Summary Result: reason, result_1 y result_2")

	-> Reagrupar en tres variables Inspection Summary Result: reason, result_1 y result_2


In [28]:
df['inspection_summary_result'] = df['inspection_summary_result'].astype('str')

In [29]:
df_3 = pd.DataFrame(df.inspection_summary_result.str.split('_-_',1).tolist(), columns= ['reason', 'result'])

In [30]:
df_3['result'] = df_3['result'].astype('str')

In [31]:
df_4 = pd.DataFrame(df_3.result.str.split(';_',1).tolist(), columns = ['result_1', 'result_2'])

In [32]:
df_3 = df_3.drop(df_3.columns[[1]], axis=1) 

In [33]:
df = pd.concat([df, df_3, df_4], axis=1)

In [34]:
df = df.drop(df.columns[[33]], axis = 1) #Eliminar inspection_summary_result

In [35]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 51306 entries, 0 to 52023
Data columns (total 48 columns):
 #   Column                                       Non-Null Count  Dtype  
---  ------                                       --------------  -----  
 0   center_name                                  39371 non-null  object 
 1   legal_name                                   39371 non-null  object 
 2   building                                     39360 non-null  object 
 3   street                                       39371 non-null  object 
 4   zipcode                                      39321 non-null  float64
 5   phone                                        39325 non-null  object 
 6   permit_number                                35549 non-null  float64
 7   permit_expiration                            39371 non-null  object 
 8   age_range                                    37584 non-null  object 
 9   maximum_capacity                             39371 non-null  float64
 10

In [36]:
print("\t-> A la variable reason la hacemos dummy, es decir, initial annual inspection es 1 y en otro caso es cero")

	-> A la variable reason la hacemos dummy, es decir, initial annual inspection es 1 y en otro caso es cero


In [37]:
df.reason.value_counts(dropna=False)

initial_annual_inspection                       25075
nan                                             11690
compliance_inspection_of_open_violations         6857
monitoring_inspection_non-routine                5223
NaN                                              2386
lead_based_paint_or_lead_in_water_inspection       75
Name: reason, dtype: int64

In [40]:
df['initial_annual_inspection'] = df.reason.apply(lambda x: 1 if x == "initial_annual_inspection" else 0)

In [41]:
df.initial_annual_inspection.value_counts(dropna=False)

0    26231
1    25075
Name: initial_annual_inspection, dtype: int64

In [42]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 51306 entries, 0 to 52023
Data columns (total 49 columns):
 #   Column                                       Non-Null Count  Dtype  
---  ------                                       --------------  -----  
 0   center_name                                  39371 non-null  object 
 1   legal_name                                   39371 non-null  object 
 2   building                                     39360 non-null  object 
 3   street                                       39371 non-null  object 
 4   zipcode                                      39321 non-null  float64
 5   phone                                        39325 non-null  object 
 6   permit_number                                35549 non-null  float64
 7   permit_expiration                            39371 non-null  object 
 8   age_range                                    37584 non-null  object 
 9   maximum_capacity                             39371 non-null  float64
 10

In [43]:
df = df.drop(df.columns[[45]], axis=1) #Eliminamos la variable reason

In [44]:
print("\t-> Creamos dummies a las variables result_1 y result_2")

	-> Creamos dummies a las variables result_1 y result_2


In [45]:
dummies = ["result_1", "result_2"]
df_2 = df[[i for i in df.keys() if i not in dummies]]
df_3 = pd.get_dummies(df[dummies])

In [46]:
df = pd.concat([df_2.reset_index(drop=True), df_3], axis=1)
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 51889 entries, 0 to 52023
Data columns (total 56 columns):
 #   Column                                               Non-Null Count  Dtype  
---  ------                                               --------------  -----  
 0   center_name                                          39371 non-null  object 
 1   legal_name                                           39371 non-null  object 
 2   building                                             39360 non-null  object 
 3   street                                               39371 non-null  object 
 4   zipcode                                              39321 non-null  float64
 5   phone                                                39325 non-null  object 
 6   permit_number                                        35549 non-null  float64
 7   permit_expiration                                    39371 non-null  object 
 8   age_range                                            37584 non-nul

In [47]:
print("\t-> Creamos variables de año, mes y día a partir de Inspection date")

	-> Creamos variables de año, mes y día a partir de Inspection date


In [48]:
df['inspection_date'] = pd.to_datetime(df.inspection_date, format = '%m/%d/%Y')

In [49]:
df['inspection_year'] = df['inspection_date'].dt.year

In [50]:
df['inspection_month_name'] = df['inspection_date'].dt.month_name()

In [51]:
df['inspection_day_name'] = df['inspection_date'].dt.day_name()

In [52]:
print("\t-> Eliminamos días festivos, sábado y domingo ")

	-> Eliminamos días festivos, sábado y domingo 


In [53]:
df = df.drop(df.loc[df['inspection_day_name']== 'Saturday'].index)

In [54]:
df = df.drop(df.loc[df['inspection_day_name']== 'Sunday'].index)

In [55]:
print("\t-> Poner como primer columna center_id e inspection_date")

	-> Poner como primer columna center_id e inspection_date


In [56]:
df.rename(columns={'day_care_id':'center_id'}, inplace=True)

In [57]:
def order(frame,var): 
    varlist =[w for w in frame.columns if w not in var] 
    frame = frame[var+varlist] 
    return frame

In [58]:
df = order(df,['center_id', 'inspection_date'])

In [59]:
print("\t-> Ordenamos la base por year, month y day en forma descendente")

	-> Ordenamos la base por year, month y day en forma descendente


In [60]:
df.sort_values(['inspection_date'], ascending=[False], inplace=True)

In [61]:
print("\t-> Creamos dummy = 1 si existió violación")

	-> Creamos dummy = 1 si existió violación


In [62]:
df.violation_category.value_counts(dropna=False)

NaN                     28011
general                 12136
critical                 8338
public_health_hazard     3396
Name: violation_category, dtype: int64

In [63]:
df['violation'] = df['violation_category'].apply(lambda x: not pd.isnull(x))

In [64]:
df['violation'] = df['violation'].apply(lambda x: 1 if x == True else 0)

In [65]:
df.violation.value_counts(dropna=False)

0    28011
1    23870
Name: violation, dtype: int64

In [66]:
print("\t-> Creamos dummy = 1 si existió violación y es un problema de salud pública")

	-> Creamos dummy = 1 si existió violación y es un problema de salud pública


In [67]:
df['public_hazard'] = df['violation_category'].apply(lambda x: 1 if x == 'public_health_hazard' else 0)

In [68]:
df.public_hazard.value_counts(dropna=False)

0    48485
1     3396
Name: public_hazard, dtype: int64

In [69]:
print("\t-> Creamos la variable violaciones_hist_salud_publica: Número de violaciones de salud pública históricas (2016-2019) por centro")

	-> Creamos la variable violaciones_hist_salud_publica: Número de violaciones de salud pública históricas (2016-2019) por centro


In [70]:
df['violaciones_hist_salud_publica'] = df.public_hazard[(df.inspection_year != 2020)]

In [71]:
df_4 = df.groupby('center_id').violaciones_hist_salud_publica.sum().reset_index()

In [72]:
df = pd.merge(left=df,right=df_4, how='left', left_on='center_id', right_on='center_id')

In [73]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 51881 entries, 0 to 51880
Data columns (total 63 columns):
 #   Column                                               Non-Null Count  Dtype         
---  ------                                               --------------  -----         
 0   center_id                                            39363 non-null  object        
 1   inspection_date                                      39307 non-null  datetime64[ns]
 2   center_name                                          39363 non-null  object        
 3   legal_name                                           39363 non-null  object        
 4   building                                             39352 non-null  object        
 5   street                                               39363 non-null  object        
 6   zipcode                                              39313 non-null  float64       
 7   phone                                                39317 non-null  object        
 

In [74]:
df = df.drop(df.columns[[61]], axis=1) #Eliminamos la variable repetida

In [75]:
df.rename(columns={'violaciones_hist_salud_publica_y':'violaciones_hist_salud_publica'}, inplace=True)

In [76]:
print("\t-> Creamos la variable violaciones_2019_salud_publica: Número de violaciones de salud pública en el 2019 por centro")

	-> Creamos la variable violaciones_2019_salud_publica: Número de violaciones de salud pública en el 2019 por centro


In [77]:
df['violaciones_2019_salud_publica'] = df.public_hazard[(df.inspection_year == 2019)]

In [78]:
df_5 = df.groupby('center_id').violaciones_2019_salud_publica.sum().reset_index()

In [79]:
df = pd.merge(left=df,right=df_5, how='left', left_on='center_id', right_on='center_id')

In [80]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 51881 entries, 0 to 51880
Data columns (total 64 columns):
 #   Column                                               Non-Null Count  Dtype         
---  ------                                               --------------  -----         
 0   center_id                                            39363 non-null  object        
 1   inspection_date                                      39307 non-null  datetime64[ns]
 2   center_name                                          39363 non-null  object        
 3   legal_name                                           39363 non-null  object        
 4   building                                             39352 non-null  object        
 5   street                                               39363 non-null  object        
 6   zipcode                                              39313 non-null  float64       
 7   phone                                                39317 non-null  object        
 

In [81]:
df = df.drop(df.columns[[62]], axis=1) #Eliminamos la variable repetida

In [82]:
df.rename(columns={'violaciones_2019_salud_publica_y':'violaciones_2019_salud_publica'}, inplace=True)

In [83]:
print("\t-> Creamos la variable violaciones_hist_criticas: Número de violaciones críticas históricas anteriores (2016-2019) por centro")

	-> Creamos la variable violaciones_hist_criticas: Número de violaciones críticas históricas anteriores (2016-2019) por centro


In [84]:
df['violation_critical'] = df['violation_category'].apply(lambda x: 1 if x == 'critical' else 0)

In [85]:
df['violaciones_hist_criticas'] = df.violation_critical[(df.inspection_year != 2020)]

In [86]:
df_6 = df.groupby('center_id').violaciones_hist_criticas.sum().reset_index()

In [87]:
df = pd.merge(left=df,right=df_6, how='left', left_on='center_id', right_on='center_id')

In [88]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 51881 entries, 0 to 51880
Data columns (total 66 columns):
 #   Column                                               Non-Null Count  Dtype         
---  ------                                               --------------  -----         
 0   center_id                                            39363 non-null  object        
 1   inspection_date                                      39307 non-null  datetime64[ns]
 2   center_name                                          39363 non-null  object        
 3   legal_name                                           39363 non-null  object        
 4   building                                             39352 non-null  object        
 5   street                                               39363 non-null  object        
 6   zipcode                                              39313 non-null  float64       
 7   phone                                                39317 non-null  object        
 

In [89]:
df = df.drop(df.columns[[64]], axis=1) #Eliminamos la variable repetida

In [90]:
df.rename(columns={'violaciones_hist_criticas_y':'violaciones_hist_criticas'}, inplace=True)

In [91]:
print("\t-> Creamos la variable violaciones_2019_criticas: Número de violaciones críticas en el 2019 por centro")

	-> Creamos la variable violaciones_2019_criticas: Número de violaciones críticas en el 2019 por centro


In [92]:
df['violaciones_2019_criticas'] = df.violation_critical[(df.inspection_year == 2019)]

In [93]:
df_7 = df.groupby('center_id').violaciones_2019_criticas.sum().reset_index()

In [94]:
df = pd.merge(left=df,right=df_7, how='left', left_on='center_id', right_on='center_id')

In [95]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 51881 entries, 0 to 51880
Data columns (total 67 columns):
 #   Column                                               Non-Null Count  Dtype         
---  ------                                               --------------  -----         
 0   center_id                                            39363 non-null  object        
 1   inspection_date                                      39307 non-null  datetime64[ns]
 2   center_name                                          39363 non-null  object        
 3   legal_name                                           39363 non-null  object        
 4   building                                             39352 non-null  object        
 5   street                                               39363 non-null  object        
 6   zipcode                                              39313 non-null  float64       
 7   phone                                                39317 non-null  object        
 

In [96]:
df = df.drop(df.columns[[65]], axis=1) #Eliminamos la variable repetida

In [97]:
df.rename(columns={'violaciones_2019_criticas_y':'violaciones_2019_criticas'}, inplace=True)

In [98]:
print("\t-> Creamos la variable ratio_violaciones_hist: Número de inspecciones en total de primera vez que resultaron en violación crítica o de salud pública/ número de inspecciones de primera vez por centro")

	-> Creamos la variable ratio_violaciones_hist: Número de inspecciones en total de primera vez que resultaron en violación crítica o de salud pública/ número de inspecciones de primera vez por centro


In [99]:
df_8 = df.loc[df['inspection_year'] != 2020]

In [100]:
df_8 = df[df.violation_category.isin(['critical', 'public_health_hazard']) & df['initial_annual_inspection']==1]

In [101]:
df_9 = df_8.groupby('center_id').initial_annual_inspection.sum().reset_index()

In [102]:
df_10 = df.groupby('center_id').initial_annual_inspection.sum().reset_index()

In [103]:
df_11 = pd.merge(left=df_10,right=df_9, how='left', left_on='center_id', right_on='center_id')

In [104]:
df_11['ratio_violaciones_hist'] = df_11['initial_annual_inspection_y'] / df_11['initial_annual_inspection_x']

In [105]:
df = pd.merge(left=df,right=df_11, how='left', left_on='center_id', right_on='center_id')

In [106]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 51881 entries, 0 to 51880
Data columns (total 69 columns):
 #   Column                                               Non-Null Count  Dtype         
---  ------                                               --------------  -----         
 0   center_id                                            39363 non-null  object        
 1   inspection_date                                      39307 non-null  datetime64[ns]
 2   center_name                                          39363 non-null  object        
 3   legal_name                                           39363 non-null  object        
 4   building                                             39352 non-null  object        
 5   street                                               39363 non-null  object        
 6   zipcode                                              39313 non-null  float64       
 7   phone                                                39317 non-null  object        
 

In [107]:
df = df.drop(df.columns[[66,67]], axis=1) #Eliminamos variables que no necesitamos 

In [108]:
print("\t-> Creamos la variable ratio_violaciones_2019: Número de inspecciones en total de primera vez que resultaron en violación crítica o de salud pública en el 2019 / número de inspecciones de primera vez por centro")

	-> Creamos la variable ratio_violaciones_2019: Número de inspecciones en total de primera vez que resultaron en violación crítica o de salud pública en el 2019 / número de inspecciones de primera vez por centro


In [109]:
df_12 = df.loc[df['inspection_year'] == 2019]

In [110]:
df_12 = df[df.violation_category.isin(['critical', 'public_health_hazard']) & df['initial_annual_inspection']==1]

In [111]:
df_13 = df_12.groupby('center_id').initial_annual_inspection.sum().reset_index()

In [112]:
df_14 = pd.merge(left=df_10,right=df_13, how='left', left_on='center_id', right_on='center_id')

In [113]:
df_14['ratio_violaciones_2019'] = df_14['initial_annual_inspection_y'] / df_14['initial_annual_inspection_x']

In [114]:
df = pd.merge(left=df,right=df_14, how='left', left_on='center_id', right_on='center_id')

In [115]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 51881 entries, 0 to 51880
Data columns (total 70 columns):
 #   Column                                               Non-Null Count  Dtype         
---  ------                                               --------------  -----         
 0   center_id                                            39363 non-null  object        
 1   inspection_date                                      39307 non-null  datetime64[ns]
 2   center_name                                          39363 non-null  object        
 3   legal_name                                           39363 non-null  object        
 4   building                                             39352 non-null  object        
 5   street                                               39363 non-null  object        
 6   zipcode                                              39313 non-null  float64       
 7   phone                                                39317 non-null  object        
 

In [116]:
df = df.drop(df.columns[[67,68]], axis=1) #Eliminamos variables que no necesitamos 